<a href="https://colab.research.google.com/github/HiranmaiKaredla/PatriotHacks/blob/main/Text_to_CSV_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

In [2]:
#!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv

In [17]:
import pandas as pd

df = pd.read_csv("/content/apartments_for_rent_classified_10K.csv", encoding="latin-1", sep=";")
print(df.shape)
print(df.columns.tolist())

(10000, 22)
['id', 'category', 'title', 'body', 'amenities', 'bathrooms', 'bedrooms', 'currency', 'fee', 'has_photo', 'pets_allowed', 'price', 'price_display', 'price_type', 'square_feet', 'address', 'cityname', 'state', 'latitude', 'longitude', 'source', 'time']


In [18]:
df.head(5)

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668626895,housing/rent/apartment,"Studio apartment 2nd St NE, Uhland Terrace NE,...","This unit is located at second St NE, Uhland T...",NaN,NaN,0.0,USD,No,Thumbnail,...,$790,Monthly,101,NaN,Washington,DC,38.9057,-76.9861,RentLingo,1577359415
1,5664597177,housing/rent/apartment,Studio apartment 814 Schutte Road,"This unit is located at 814 Schutte Road, Evan...",NaN,NaN,1.0,USD,No,Thumbnail,...,$425,Monthly,106,814 Schutte Rd,Evansville,IN,37.9680,-87.6621,RentLingo,1577017063
2,5668626833,housing/rent/apartment,"Studio apartment N Scott St, 14th St N, Arling...","This unit is located at N Scott St, 14th St N,...",NaN,1.0,0.0,USD,No,Thumbnail,...,"$1,390",Monthly,107,NaN,Arlington,VA,38.8910,-77.0816,RentLingo,1577359410
3,5659918074,housing/rent/apartment,Studio apartment 1717 12th Ave,"This unit is located at 1717 12th Ave, Seattle...",NaN,1.0,0.0,USD,No,Thumbnail,...,$925,Monthly,116,1717 12th Avenue,Seattle,WA,47.6160,-122.3275,RentLingo,1576667743
4,5668626759,housing/rent/apartment,"Studio apartment Washington Blvd, N Cleveland ...","This unit is located at Washington Blvd, N Cle...",NaN,NaN,0.0,USD,No,Thumbnail,...,$880,Monthly,125,NaN,Arlington,VA,38.8738,-77.1055,RentLingo,1577359401


In [4]:
#%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

In [19]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///properties.db")
df.to_sql("properties", engine, index=False)

10000

In [20]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM properties WHERE cityname = 'Seattle' limit 3;")

sqlite
['properties']


"[(5659918074, 'housing/rent/apartment', 'Studio apartment 1717 12th Ave', 'This unit is located at 1717 12th Ave, Seattle, 98122, WAMonthly rental rates range from $925We have studio units available for rent', None, 1.0, 0.0, 'USD', 'No', 'Thumbnail', None, 925, '$925', 'Monthly', 116, '1717 12th Avenue', 'Seattle', 'WA', 47.616, -122.3275, 'RentLingo', 1576667743), (5668634310, 'housing/rent/apartment', 'Studio apartment 6921 Roosevelt Way NE', 'This unit is located at 6921 Roosevelt Way NE, Seattle, 98115, WAMonthly rental rates range from $1195We have studio units available for rent', None, 1.0, 0.0, 'USD', 'No', 'Thumbnail', 'Cats,Dogs', 1195, '$1,195', 'Monthly', 223, '6921 Roosevelt Way NE', 'Seattle', 'WA', 47.616, -122.3275, 'RentLingo', 1577359975), (5668640879, 'housing/rent/apartment', 'Studio apartment 714 N. 95th St.', 'This unit is located at 714 N. 95th St., Seattle, 98103, WAMonthly rental rates range from $1150We have studio units available for rent', None, 1.0, 0.0, 

In [21]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [22]:
agent_executor.invoke({"input": "Give the properties in Seattle with price under $1000"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


properties
Invoking: `sql_db_schema` with `{'table_names': 'properties'}`



CREATE TABLE properties (
	id BIGINT, 
	category TEXT, 
	title TEXT, 
	body TEXT, 
	amenities TEXT, 
	bathrooms FLOAT, 
	bedrooms FLOAT, 
	currency TEXT, 
	fee TEXT, 
	has_photo TEXT, 
	pets_allowed TEXT, 
	price BIGINT, 
	price_display TEXT, 
	price_type TEXT, 
	square_feet BIGINT, 
	address TEXT, 
	cityname TEXT, 
	state TEXT, 
	latitude FLOAT, 
	longitude FLOAT, 
	source TEXT, 
	time BIGINT
)

/*
3 rows from properties table:
id	category	title	body	amenities	bathrooms	bedrooms	currency	fee	has_photo	pets_allowed	price	price_display	price_type	square_feet	address	cityname	state	latitude	longitude	source	time
5668626895	housing/rent/apartment	Studio apartment 2nd St NE, Uhland Terrace NE, Washington, DC 20002	This unit is located at second St NE, Uhland Terrace NE, Washington, DC 20002, Washington, 20002, DC	None	None	0.0

{'input': 'Give the properties in Seattle with price under $1000',
 'output': 'The property in Seattle with a price under $1000 is a studio apartment located at 1717 12th Ave, Seattle, WA. The monthly rental rate for this property is $925.'}